## MLflow Configuration

In [1]:
#!pip install mlflow
#!pip install --upgrade jinja2
#!pip install --upgrade Flask
#!pip install setuptools

In [2]:
## Import de librairie
import pandas as pd
import mlflow
from mlflow import MlflowClient
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from pprint import pprint
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

c:\Users\j_aka\anaconda3\envs\statistique\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [3]:
## Connecxion au serveur MLFlow
client= MlflowClient(tracking_uri="http://127.0.0.1:8080")
## Configuration 
mlflow.set_tracking_uri("http://127.0.0.1:8080")


In [4]:
# Récupération de toutes les expériences MLflow
all_experiments = client.search_experiments()
# Affichage des expériences 
print(all_experiments)

[<Experiment: artifact_location='mlflow-artifacts:/606383372813198707', creation_time=1760806668542, experiment_id='606383372813198707', last_update_time=1760806668542, lifecycle_stage='active', name='TREE_Models', tags={'mlflow.experimentKind': 'custom_model_development'}>, <Experiment: artifact_location='mlflow-artifacts:/389458555972686615', creation_time=1760804935652, experiment_id='389458555972686615', last_update_time=1760804935652, lifecycle_stage='active', name='RLG_Models', tags={'mlflow.experimentKind': 'custom_model_development'}>, <Experiment: artifact_location='mlflow-artifacts:/668826699846372727', creation_time=1760799130123, experiment_id='668826699846372727', last_update_time=1760799130123, lifecycle_stage='active', name='RDF_Models', tags={'mlflow.experimentKind': 'custom_model_development'}>, <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1760798921200, experiment_id='0', last_update_time=1760798921200, lifecycle_stage='active', name='Default', 

In [5]:
# Recherche des expériences Mlflow Spécifique.
client.search_experiments(filter_string="name = 'RLG_Models'")

[<Experiment: artifact_location='mlflow-artifacts:/389458555972686615', creation_time=1760804935652, experiment_id='389458555972686615', last_update_time=1760804935652, lifecycle_stage='active', name='RLG_Models', tags={'mlflow.experimentKind': 'custom_model_development'}>]

## Chargement du fichier 

In [6]:
# Définition du chemin vers le fichier de  données 
data_dir = r"C:\Users\j_aka\Desktop\mlops\credit_risk_dataset.csv"
# Chargement de la données 
df = pd.read_csv(data_dir, delimiter=",") 
#Affichage des 3 premières lignes 
print(df.head(3).to_string())

   person_age  person_income person_home_ownership  person_emp_length loan_intent loan_grade  loan_amnt  loan_int_rate  loan_status  loan_percent_income cb_person_default_on_file  cb_person_cred_hist_length
0          22          59000                  RENT              123.0    PERSONAL          D      35000          16.02            1                 0.59                         Y                           3
1          21           9600                   OWN                5.0   EDUCATION          B       1000          11.14            0                 0.10                         N                           2
2          25           9600              MORTGAGE                1.0     MEDICAL          C       5500          12.87            1                 0.57                         N                           3


## Regression logistic model

In [ ]:
# Définition des Features et de la cible 
target_col = 'loan_status'
X = df.drop(columns=['loan_status'])    
y = df['loan_status']

# Split des données 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Colonne numérique et colonne catégorielle 
num_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_features = X.select_dtypes(include=["object", "category"]).columns.tolist()

#Encoder et Gérer les données Nan 
preprocessor = ColumnTransformer(
    transformers= [
        ("num_features", SimpleImputer(strategy="mean"), num_features),
        ("cat_features", OneHotEncoder(handle_unknown="ignore"),cat_features)
    ]
)


# Hyperparamètre du modèle
#params = {
   # "max_iter": 1000,
#}
# Nom du modèle : 
mlflow.set_experiment("RLG_Models")
# Nom du run pour cette itération d'entraînement
run_name = "RLG_Run"
# les artefacts du modèle
artifact_path = "RLG_artefacts"
# Fermer tout run précédent
mlflow.end_run()

with mlflow.start_run(run_name=run_name):
    # Tag de version du modèle
    mlflow.set_tag("version", "v1.0")

# Encodage et entrainement du modèle 
    pipeline_fr = Pipeline(steps=[
    ("processor", preprocessor),
    ("model", LogisticRegression(max_iter=3000))
   ])
    
# Entrainement du modèle
    pipeline_fr.fit(X_train,y_train)

# Prédiction du modèle
    y_proba = pipeline_fr.predict_proba(X_val)[:, 1]
    y_pred = (y_proba >= 0.3).astype(int)

# Calcul des métriques
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    metrics = {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1
    }

    mlflow.log_param("max_iter",1000)
    mlflow.log_metrics(metrics)
    mlflow.sklearn.log_model(
        sk_model= pipeline_fr, 
        input_example=X_val, 
        artifact_path=artifact_path
    ) 

    print("✅ Modèle entraîné et logué dans MLflow avec succès.")
    print("\nRapport de classification :\n", classification_report(y_val, y_pred))
    print("\nMatrice de confusion :\n", confusion_matrix(y_val, y_pred))
    print("📊 Métriques :", metrics)

c:\Users\j_aka\anaconda3\envs\statistique\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 3000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=3000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
2025/10/19 13:13:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Users\j_aka\anaconda3\envs\statistique\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be enco

✅ Modèle entraîné et logué dans MLflow avec succès.

Rapport de classification :
               precision    recall  f1-score   support

           0       0.86      0.96      0.91      5072
           1       0.75      0.46      0.57      1445

    accuracy                           0.85      6517
   macro avg       0.81      0.71      0.74      6517
weighted avg       0.84      0.85      0.83      6517


Matrice de confusion :
 [[4850  222]
 [ 776  669]]
📊 Métriques : {'accuracy': 0.8468620530919134, 'precision': 0.7508417508417509, 'recall': 0.4629757785467128, 'f1': 0.5727739726027398}
🏃 View run RLG_Run at: http://127.0.0.1:8080/#/experiments/389458555972686615/runs/a58f24bbc7534a6a88d4f4ae7cc12c9d
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/389458555972686615
